In [1]:
import pandas as pd
import numpy as np
import pickle

import nltk
# from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

In [2]:
data= pd.read_csv("GARDINERS_SENTENCES (1).csv")
data.head()

,Gardiner
0,D21Q3D36F4D36L2X1S19S29U23T21X1G17D21Z1R8U36S4...
1,M17A26S34A1G43A1Z3N17N23A1Z2BS29V4X1S29N35D2Z1...
2,W24V31V22F34N35M23X1N35G17R8O6X1O1Z1D21Z1O47Z1...
3,G35F34F34F34D2Z1A17U6D21M17M17X1N23F20P28Z2G36...
4,M17G43D4N35M17M40O34O1Z1G17V28W14X1O34M23X1N35...


In [3]:
data.Gardiner[0]

'D21Q3D36F4D36L2X1S19S29U23T21X1G17D21Z1R8U36S4F29X1X1B1V30X1U23D58G43N25O49M17U1F39A1M17M17A1D21E16A40G39D21N35Q3G43X1Z3A1D4N35G39X1V13N35Z1M17B1I10D46'

In [4]:
data=data.dropna()
data.isna().sum()

Gardiner    0
dtype: int64

In [5]:
data_path='./'

In [6]:
with open(data_path+"train.pickle", "rb") as f:
    (X,y,sizes) = pickle.load(f)

In [7]:
sentences=[]
for sentence in data.Gardiner[:]:
    codes=[]
    code=''
    first_letter=False
    for i in sentence:
        if i.isupper() :
            if first_letter:
                codes.append(code)
                code=''
            else:
                first_letter=True
        code+=i
    sentences.append(codes)
            

In [8]:
# sentences[0]
all_codes=list(sizes.keys())

In [9]:
new_sentences=[]
for sentence in sentences:
    doesnt_exists=False
    for code in sentence:
        if code not in all_codes:
            doesnt_exists=True
    if doesnt_exists == False:
        new_sentences.append(sentence)
            
            

In [10]:
final_data=[]
for sent in new_sentences:
    if len(sent) > 2:
        final_data.append(sent)

In [17]:
# final_data

In [11]:
len(final_data)

2179

In [12]:
#saving data as pickle
with open("lm_sentences.pickle", "wb") as f:
    pickle.dump((final_data),f)

In [13]:
def train_model(model,sentences,n=2):
    # Count frequency of co-occurance
    if n==3:
        for sentence in sentences:
            for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
                model[(w1, w2)][w3] += 1
        # Let's transform the counts to probabilities
        for w1_w2 in model:
            total_count = float(sum(model[w1_w2].values()))
            for w3 in model[w1_w2]:
                model[w1_w2][w3] /= total_count
    elif n==2:
        #bigram
        for sentence in sentences:
            for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
                model[(w1)][w2] += 1
        for w1_w2 in model:
            total_count = float(sum(model[w1_w2].values()))
            for w3 in model[w1_w2]:
                model[w1_w2][w3] /= total_count
    return model

In [14]:
def lm_next(model,prev):
    pred = dict( eval('model'+ str(prev)))
    next_scores = sorted(pred.items(), key=lambda item: item[1],reverse=True)
    out = dict(next_scores)
    if len(list(out.keys()))==0:
        out ={'None':0}
    return out

In [26]:
# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

In [27]:
model = train_model(model,sentences,n=3)

In [19]:
# sentences

In [29]:
prev=['G1', 'G1'] 
prediction=lm_next(model,prev)
# prediction

In [30]:
import dill as pickle

with open("language_model_sent.pkl", "wb") as f:
    pickle.dump(model,f)

In [65]:
# sentences[1]